In [1]:
import os
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


class CoronarySmallDataset(Dataset):
    def __init__(self, image_dir, keypoint_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.keypoint_dir = keypoint_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.input = os.listdir(image_dir)

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        SIZE = 256
        img_path = os.path.join(self.image_dir, self.input[idx])
        keypoint_path = os.path.join(self.keypoint_dir, self.input[idx])
        mask_path = os.path.join(self.mask_dir, self.input[idx])
        

        image = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        image = cv2.resize(image, (SIZE, SIZE), interpolation=cv2.INTER_NEAREST)

        keypoint = cv2.imread(keypoint_path, cv2.IMREAD_UNCHANGED)
        keypoint = cv2.resize(keypoint, (SIZE, SIZE), interpolation=cv2.INTER_NEAREST)

        output = np.stack((image, keypoint), axis=2)

        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        mask = cv2.resize(mask, (SIZE, SIZE), interpolation=cv2.INTER_NEAREST)
        
        if self.transform:
            output = self.transform(output)
            mask = torch.tensor(mask, dtype=torch.long)
        
        return output, mask


transform = transforms.Compose([
    transforms.ToTensor()
])

train_image_dir = '../images/images_train_original/input'
train_keypoint_dir = '../images/images_train_original/keypoints'
train_mask_dir = '../images/images_train_original/output'
val_image_dir = '../images/images_val/input'
val_keypoint_dir = '../images/images_val/keypoints'
val_mask_dir = '../images/images_val/output'
test_image_dir = '../images/images_test/input'
test_keypoint_dir = '../images/images_test/keypoints'
test_mask_dir = '../images/images_test/output'

train_dataset = CoronarySmallDataset(train_image_dir, train_keypoint_dir, train_mask_dir, transform=transform)
val_dataset = CoronarySmallDataset(val_image_dir, val_keypoint_dir, val_mask_dir, transform=transform)
test_dataset = CoronarySmallDataset(test_image_dir, test_keypoint_dir, test_mask_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=6, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=6, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=6, shuffle=False)


In [2]:
from converter_RGB import convert_int_to_RGB
from large_RGB_model import UNet


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet()
model = model.to(device)

model.load_state_dict(torch.load('model_kp_1.pth', map_location=device))
model.eval()

def show_image(type, image_name):
    dir = f"..\images\images_test\{type}"
    print(dir)
    if type == 'output':
        img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
        # img = img[:, :, 2]
        print("Oryginalnie: ", np.unique(img))
        
        # print("Po resize: NEAREST\t  ", np.unique(cv2.resize(img, (256, 256), interpolation=cv2.INTER_NEAREST)))
        # cv2.imshow('INTER_NEAREST', cv2.resize(convert_int_to_RGB(cv2.resize(img, (256, 256), interpolation=cv2.INTER_NEAREST)), (512, 512)))
        # print("Po resize: NEAREST_EXACT  ", np.unique(cv2.resize(img, (256, 256), interpolation=cv2.INTER_NEAREST_EXACT)))
        # cv2.imshow('INTER_NEAREST_EXACT', cv2.resize(convert_int_to_RGB(cv2.resize(img, (256, 256), interpolation=cv2.INTER_NEAREST_EXACT)), (512, 512)))
        
        print("Po resize:  ", np.unique(cv2.resize(img, (256, 256))))
        # print(img)
        img = convert_int_to_RGB(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    else:
        img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
        img = img * 7
    # img = cv2.resize(img, (256, 256))
    # np.set_printoptions(threshold=np.inf)
    cv2.imshow(type, img)

def predict(image_name):
    dir = '..\images\images_test\input'
    img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
    img = cv2.resize(img, (256, 256))
    img = transforms.ToTensor()(img).unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model.predict(img)
        pred = pred.squeeze().cpu().numpy()
    print("Z sieci:    ", np.unique(pred))

    pred_image = convert_int_to_RGB(pred)
    pred_image = cv2.resize(pred_image, (512, 512))
    pred_image = cv2.cvtColor(pred_image, cv2.COLOR_BGR2RGB)
    cv2.imshow('pred', pred_image)
    cv2.waitKey(0)
   

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_28260\934138035.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_kp_1.pth', map_loca

In [18]:
# image_name = "131aedfhs6pnf1fvtvp49mhdb2fucqzo22_29.png"
# image_name = "131aedfhs6pnf1fvtvp49mld7mqexnz322_36.png"
# image_name = "131aedfhs6pnf1fvtvp49mia892s56cf22_28.png"
# image_name = "131aedfhs6pnf1fvtvp49juwu7plj9dv22_40.png"
# image_name = "13c2ur549vohc0jat2dvu3xs7q1_18.png"
image_name = "131aedfhs6pnf1fvtvp49mi8hml81goj22_37.png"

# treningowe
# image_name = "13c2ur549vohc0jat2e6y42xk11_31.png"
# image_name = "131aedfhs6pnf1fvtvp49jstof8iams322_36.png"
# image_name = "131aedfhs6pnf1fvtvp49mff2plx3rdw22_24.png"
show_image("input", image_name)
show_image("output", image_name)
predict(image_name)

..\images\images_test\input
..\images\images_test\output
Oryginalnie:  [ 0  5  6  7  8  9 10 11 13 15 16 17]
Po resize:   [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
Z sieci:     [ 0  5  6  7  8  9 13 15 17 18]


In [3]:
from sklearn.metrics import precision_score, recall_score, f1_score


def compute_accuracy(model, data_loader, device):
    model.eval()
    correct = 0
    correct_clear = 0
    total = 0
    total_clear = 0

    with torch.no_grad():
        for images, masks in data_loader:
            images = images.to(device)
            masks = masks.to(device)

            preds = model.predict(images)
            
            correct += (preds == masks).sum().item()
            correct_clear += (preds[masks != 0] == masks[masks != 0]).sum().item()
            total += masks.numel()
            total_clear += masks[masks != 0].numel()

    accuracy = correct / total
    accuracy_clear = correct_clear / total_clear
    return accuracy, accuracy_clear

In [4]:
def compute_precision_recall_f1(model, data_loader, device, num_classes=28):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, masks in data_loader:
            images = images.to(device)
            masks = masks.to(device)

            preds = model.predict(images)
            
            all_preds.append(preds.view(-1).cpu().numpy())
            all_targets.append(masks.view(-1).cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)

    TP = np.zeros(num_classes)
    FP = np.zeros(num_classes)
    FN = np.zeros(num_classes)

    for i in range(num_classes):
        TP[i] = ((all_preds == i) & (all_targets == i)).sum()
        FP[i] = ((all_preds == i) & (all_targets != i)).sum()
        FN[i] = ((all_preds != i) & (all_targets == i)).sum()
    
    TP[0], FP[0], FN[0] = 0, 0, 0
    # print(TP, FP, FN)

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precision * recall / (precision + recall)
    # print(TP.sum(), FP.sum(), FN.sum())

    precision_weighted = TP.sum() / (TP.sum() + FP.sum())
    recall_weighted = TP.sum() / (TP.sum() + FN.sum())
    f1_weighted = 2 * precision_weighted * recall_weighted / (precision_weighted + recall_weighted)

    return precision, recall, f1, precision_weighted, recall_weighted, f1_weighted

In [8]:
# learning_rate = 2.4, dropout_rate = 0.2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_kp_1.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_32656\1913739539.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_kp_1.pth', map_loc

Train Accuracy: 98.31%
Train Accuracy Clear: 67.89%

Validation Accuracy: 97.64%
Validation Accuracy Clear: 55.72%


In [9]:
model.load_state_dict(torch.load('model_kp_1.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_32656\2217008298.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_kp_1.pth', map_loc

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 87.53%, Recall: 69.71%, F1: 77.61%
Class 2: Precision: 76.94%, Recall: 79.07%, F1: 77.99%
Class 3: Precision: 82.50%, Recall: 84.85%, F1: 83.66%
Class 4: Precision: 71.19%, Recall: 76.69%, F1: 73.84%
Class 5: Precision: 86.19%, Recall: 94.66%, F1: 90.23%
Class 6: Precision: 59.16%, Recall: 64.26%, F1: 61.60%
Class 7: Precision: 46.94%, Recall: 70.44%, F1: 56.33%
Class 8: Precision: 57.05%, Recall: 71.72%, F1: 63.55%
Class 9: Precision: 30.93%, Recall: 53.82%, F1: 39.29%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 66.35%, Recall: 58.58%, F1: 62.22%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 36.32%, Recall: 58.16%, F1: 44.71%
Class 16: Precision: 40.90%, Recall: 9.03%, F1: 14.79%
Class 17: Precision: 59.59%, Recall: 53.48%, F1: 56.37%
Class 18: Precisio

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_32656\1128418758.py:31: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_32656\1128418758.py:32: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [17]:
# learning_rate = 2.4, dropout_rate = 0.3   new new
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_kp_2.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_32656\2419969087.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_kp_2.pth', map_loc

Train Accuracy: 98.11%
Train Accuracy Clear: 63.81%

Validation Accuracy: 97.59%
Validation Accuracy Clear: 54.77%


In [11]:
model.load_state_dict(torch.load('model_kp_2.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_32656\1175132237.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_kp_2.pth', map_loc

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 83.48%, Recall: 61.50%, F1: 70.82%
Class 2: Precision: 66.87%, Recall: 78.56%, F1: 72.24%
Class 3: Precision: 73.16%, Recall: 73.29%, F1: 73.22%
Class 4: Precision: 62.51%, Recall: 63.10%, F1: 62.81%
Class 5: Precision: 80.53%, Recall: 89.13%, F1: 84.61%
Class 6: Precision: 48.96%, Recall: 56.48%, F1: 52.45%
Class 7: Precision: 29.62%, Recall: 41.36%, F1: 34.52%
Class 8: Precision: 40.06%, Recall: 52.43%, F1: 45.41%
Class 9: Precision: 28.04%, Recall: 22.73%, F1: 25.11%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 49.48%, Recall: 41.53%, F1: 45.16%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 22.72%, Recall: 55.68%, F1: 32.28%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 34.26%, Recall: 37.85%, F1: 35.97%
Class 18: Precision:

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_32656\1128418758.py:31: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_32656\1128418758.py:32: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [14]:
# learning_rate = 2.1, dropout_rate = 0.1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_kp_3.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_32656\3418237432.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_kp_3.pth', map_loc

Train Accuracy: 98.24%
Train Accuracy Clear: 66.35%

Validation Accuracy: 97.53%
Validation Accuracy Clear: 53.63%


In [15]:
model.load_state_dict(torch.load('model_kp_3.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_32656\2776441770.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_kp_3.pth', map_loc

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 84.33%, Recall: 73.41%, F1: 78.49%
Class 2: Precision: 67.21%, Recall: 82.77%, F1: 74.19%
Class 3: Precision: 79.01%, Recall: 69.24%, F1: 73.80%
Class 4: Precision: 77.84%, Recall: 69.37%, F1: 73.36%
Class 5: Precision: 83.41%, Recall: 97.03%, F1: 89.71%
Class 6: Precision: 50.18%, Recall: 70.07%, F1: 58.48%
Class 7: Precision: 44.09%, Recall: 73.51%, F1: 55.12%
Class 8: Precision: 69.14%, Recall: 63.10%, F1: 65.98%
Class 9: Precision: 28.52%, Recall: 56.32%, F1: 37.87%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 56.44%, Recall: 55.11%, F1: 55.77%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 36.12%, Recall: 49.36%, F1: 41.71%
Class 16: Precision: 9.09%, Recall: 0.01%, F1: 0.01%
Class 17: Precision: 67.01%, Recall: 45.17%, F1: 53.96%
Class 18: Precision:

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_32656\1128418758.py:31: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_32656\1128418758.py:32: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [19]:
# learning_rate = 2.1, dropout_rate = 0.15
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_kp_4.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_32656\1790863589.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_kp_4.pth', map_loc

Train Accuracy: 98.41%
Train Accuracy Clear: 69.69%

Validation Accuracy: 97.68%
Validation Accuracy Clear: 56.41%


In [20]:
# learning_rate = 2.5, dropout_rate = 0.4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_kp_4.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_32656\1109585907.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_kp_4.pth', map_loc

Train Accuracy: 97.70%
Train Accuracy Clear: 55.66%

Validation Accuracy: 97.25%
Validation Accuracy Clear: 48.22%


# ----------------------

In [5]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_kp_4.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_28260\1109585907.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_kp_4.pth', map_loc

Train Accuracy: 97.70%
Train Accuracy Clear: 55.66%

Validation Accuracy: 97.25%
Validation Accuracy Clear: 48.22%


In [6]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_kp_5.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_28260\2455592122.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_kp_5.pth', map_loc

Train Accuracy: 97.38%
Train Accuracy Clear: 49.25%

Validation Accuracy: 97.00%
Validation Accuracy Clear: 43.46%
